# Buscador booleano/bag-of-words no TREC-DL 2020

Aqui, é implementado um buscador booleano, que apenas leva em consideração a ocorrência ou não de cada termo da query em cada documento, independente do número de ocorrências de cada termo (abordagem bag-of-words).

## Download do dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
main_path = '/content/drive/MyDrive/Unicamp-aula-2/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Diretório já existente')

Diretório já existente


## Download de ferramentas auxiliares

In [3]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules {main_path}/pyserini

In [ ]:
!cd {main_path}/pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd {main_path}/pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

## Construção do índice invertido

In [4]:
from pyserini.analysis import Analyzer, get_lucene_analyzer

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
analyzer:Analyzer = Analyzer(get_lucene_analyzer(stemmer='porter'))

def preprocess_and_tokenize(text):
  return analyzer.analyze(text)

In [7]:

collection_path = main_path + '/collections/msmarco-passage/collection.tsv'



In [8]:
import nltk
import string

from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')  # Download stopwords if not already downloaded

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = set(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import array
import pandas as pd
from collections import defaultdict
from collections import Counter
import pickle
import os

index_path = f"{main_path}/index-tf-idf.pickle"

def load_or_build_inverted_index():
  if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      inverted_index = pickle.load(f)
  else:
    print("Building inverted index and vocabulary...")
    # set the chunk size
    chunk_size = 1000
    chunks = []
    inverted_index = dict()
    full_text = ''
    vocab = set()

    def process(row):
      tokenized_text = preprocess_and_tokenize(row[1])
      counter = Counter(tokenized_text)
      doc_length = len(tokenized_text)
      doc_id = row[0]
      for token, count in counter.items():
        if token not in stop_words:
          #Para cada token, temos 2 arrays paralelos que armazenam os documentos e as frquências dos termos
          inverted_index.setdefault(token, {"docs":array.array("L", []), "tf":array.array("f", [])})["docs"].append(int(doc_id))
          inverted_index.setdefault(token, {"docs":array.array("L", []), "tf":array.array("f", [])})["tf"].append(count/doc_length)
          vocab.add(token)

    chunk_id = 0
    n_documents = 0
    len_vocab = len(vocab)

    # iterate through the file in chunks
    for chunk in pd.read_csv(collection_path, sep='\t', header=None, chunksize=chunk_size):
      # process the chunk here
      if (chunk_id % 1000) == 0:
        print(f'Processing chunk {chunk_id}')
      for index, row in chunk.iterrows():
        process(row)
        n_documents += 1
      del(chunk)
      chunk_id += 1

    index = {"inverted_index": inverted_index, "vocab": vocab, 
             "n_documents": n_documents}

    with open(index_path, "wb") as f:
      pickle.dump(index, f)

  return index

In [12]:
index = load_or_build_inverted_index()
inverted_index = index["inverted_index"]
vocab = index["vocab"]
n_documents = index["n_documents"]

Building inverted index and vocabulary...
Processing chunk 0


KeyboardInterrupt: ignored

In [ ]:
token_to_id = {token:(i+1) for i, token in enumerate(vocab)}

In [ ]:
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
len(inverted_index)

2660662

In [ ]:
len(vocab)

2660662

In [ ]:
!head {collection_path}

0	The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
1	The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.
2	Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.
3	The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 â¦ 2-1946 under the control of the U.S. Army Corps of Engineers

## Avaliação

In [ ]:
topics_file = main_path + '/pyserini/tools/topics-and-qrels/topics.dl20.txt'
qrels_eval = main_path + '/pyserini/tools/topics-and-qrels/qrels.dl20-passage.txt'

In [ ]:
!head {topics_file}

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [ ]:
!head {qrels_eval}

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


In [ ]:
import math
import torch.nn.functional as F

def search(query):
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  query_tokens = preprocess_and_tokenize(query)
  print(query_tokens)
  n_query_tokens = len(query_tokens)
  query_tensor = torch.zeros(len(inverted_index), device=device)
  doc_to_tensor = dict()
  query_counter = Counter(query_tokens)

  for token in query_tokens:
    if token in inverted_index:
      #Calcula TF-IDF para par (termo, query)
      query_tf = query_counter[token]/n_query_tokens
      doc_ids = inverted_index[token]["docs"]
      n_docs_contain_term = len(set(doc_ids))
      idf = math.log(n_documents / n_docs_contain_term)
      query_tf_idf = query_tf * idf
      query_tensor[token_to_id[token]] = query_tf_idf

      for i, doc_id in enumerate(doc_ids):
        if doc_id not in doc_scores: #para não processar um mesmo documento novamente
          #Calcula TF-IDF para par (termo, documento)
          doc_tf = inverted_index[token]["tf"][i]
          doc_tf_idf = doc_tf * idf
          doc_tensor = torch.zeros(len(inverted_index), device=device)
          doc_tensor[token_to_id[token]] = doc_tf_idf
          doc_to_tensor[doc_id] = doc_tensor    
          
  for doc_id, doc_tensor in doc_to_tensor.items():
    doc_scores[doc_id] = F.cosine_similarity(query_tensor.unsqueeze(0), 
                                             doc_to_tensor[doc_id].unsqueeze(0), 
                                             dim=1).item()
          
  del(query_tensor)
  del(doc_to_tensor)
  return doc_scores

In [ ]:
results = search('who is aziz hashim')

['who', 'aziz', 'hashim']


OutOfMemoryError: ignored

In [ ]:
len(results)

245

In [ ]:
results
#6989780, 1305521, 4358004, 1815707, 7508059

defaultdict(int,
            {22484: 0.6599182486534119,
             68532: 0.6599182486534119,
             119291: 0.6599182486534119,
             176182: 0.6599182486534119,
             176183: 0.6599182486534119,
             226464: 0.6599182486534119,
             279227: 0.6599182486534119,
             315873: 0.6599182486534119,
             480287: 0.6599182486534119,
             596813: 0.6599182486534119,
             705402: 0.6599182486534119,
             770274: 0.6599182486534119,
             794624: 0.6599182486534119,
             821997: 0.6599182486534119,
             848943: 0.6599182486534119,
             1038342: 0.6599182486534119,
             1154757: 0.6599182486534119,
             1161432: 0.6599182486534119,
             1161439: 0.6599182486534119,
             1358683: 0.6599182486534119,
             1376556: 0.6599182486534119,
             1376558: 0.6599182486534119,
             1451842: 0.6599182486534119,
             1451844: 0.6599182486

OBS.: A lista de stopwords do Lucene Analyzer parece ser muito retrista.  Assim, para reduzir o tamanho do índice, uma alternativa seria combinar com a lista de stopwords do NLTK - ou seja, só salvar no índie invertido se não estiver na lsita de stopwords do NLTK.

In [ ]:
def get_document_by_id(id):
  result = None
  with open(collection_path, 'r') as f:
    for line in f:
      fields = line.strip().split('\t')
      doc_id = fields[0]
      if doc_id == id:
        result = fields[1]
        break

  return result

In [ ]:
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]

In [ ]:
sorted_results

[(7156982, tensor([0.7513], device='cuda:0')),
 (8726429, tensor([0.7513], device='cuda:0')),
 (8726430, tensor([0.7513], device='cuda:0')),
 (8726433, tensor([0.7513], device='cuda:0')),
 (8726434, tensor([0.7513], device='cuda:0')),
 (8726435, tensor([0.7513], device='cuda:0')),
 (8726436, tensor([0.7513], device='cuda:0')),
 (8726437, tensor([0.7513], device='cuda:0')),
 (309441, tensor([0.7513], device='cuda:0')),
 (336399, tensor([0.7513], device='cuda:0'))]

In [ ]:
get_document_by_id('336399')

'Amla may refer to: 1  Indian gooseberry, called amla in Hindi. 2  Hashim Amla, a South African cricketer of Indian descent. 3  Ahmed Amla, a South African cricketer and brother of Hashim.mla may refer to: 1  Indian gooseberry, called amla in Hindi. 2  Hashim Amla, a South African cricketer of Indian descent. 3  Ahmed Amla, a South African cricketer and brother of Hashim.'

In [ ]:
query_to_results = dict()

with open(topics_file, 'r') as f:
  for line in f:
      fields = line.strip().split('\t')
      query_id = fields[0]
      query_text = fields[1]
      results = search(query_text)
      query_to_results[int(query_id)] = sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]

with open('run.dl20.boolean.trec', 'w') as f:
  for query_id, results in query_to_results.items():
    for i, (doc_id, score) in enumerate(results):
      f.write(f'{query_id}\tQ0\t{doc_id}\t{i+1}\t{score}\tboolean\n')

['who', 'aziz', 'hashim']
['who', 'rep', 'scalis']


OutOfMemoryError: ignored

In [ ]:
!head run.dl20.boolean.trec

1030303	Q0	7156982	1	2	boolean
1030303	Q0	8726429	2	2	boolean
1030303	Q0	8726430	3	2	boolean
1030303	Q0	8726433	4	2	boolean
1030303	Q0	8726434	5	2	boolean
1030303	Q0	8726435	6	2	boolean
1030303	Q0	8726436	7	2	boolean
1030303	Q0	8726437	8	2	boolean
1030303	Q0	794624	9	1	boolean
1030303	Q0	4820481	10	1	boolean


In [ ]:
!python {main_path}/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input {qrels_eval} \
   --output qrels.dl20.trec

Done!


In [ ]:
!head qrels.dl20.trec

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


In [ ]:
!chmod 755 {main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [ ]:
!{main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   qrels.dl20.trec run.dl20.boolean.trec

map                   	all	0.1117
ndcg_cut_10           	all	0.3189
